In [1]:
from pyspark.sql import SparkSession
import spark.sparkContext as sc
spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1618243386223_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# In this exercise, we will look at doing basic manipulation of RDD.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# First exercise.
# Create a RDD from 100 numbers, tripple them and sum
def tripple(x):
    return x * 3
data = list(range(1,101))
rdd = sc.parallelize(data)
rdd = rdd.map(tripple)
first = rdd.take(1) # Verify that elements exists
print(first)
assert(first == [3])
out = rdd.sum()
print(out)
assert(out == 152150)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[3]
15150

In [35]:
# Second exercise
# Read a csv from s3 and split the csv file.

def parseRow(line):
    elements = line.split(",")
    return elements
    
rdd = rdd.flatMap(parseRow)
rdd.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['1', '200', '2', '500', '3', '-99', '4', '99', '5', '600']

In [45]:
# Third exercise
# Read a csv from s3. Find users with less score than zero, and return their ids.

def parseRow(line):
    elements = line.split(",")
    return elements
def filterPoor(x):
    return int(x[1]) < 0

rdd = sc.textFile("s3://link-workshops/example_data.csv")
rdd = rdd.map(parseRow).filter(filterPoor).map(lambda x: int(x[0]))
less_than_zero = rdd.collect()
print(less_than_zero)
assert(len(less_than_zero) == 1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[3]

In [50]:
# Fourth exercise.
# Take a list of values, and translate it through a lookup table.
# But to save time, broadcast a read only variable first.

countries  = {"sv": "sweden", "us": "usa"}
data = ["sv","us"]
broadcast_countries = spark.sparkContext.broadcast(countries)
def convert(code):
    return broadcast_countries.value[code]

rdd = spark.sparkContext.parallelize(data)
rdd = rdd.map(convert)
res = rdd.collect()
print(res)
assert(res[0] == "sweden")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['sweden', 'usa']

In [62]:
# Fifth exercise.
# Create an accumulator and implement a custom sum operation.
# Then sum the scores of the example_data.csv

accum=sc.accumulator(0)
rdd = sc.textFile("s3://link-workshops/example_data.csv")
rdd.map(parseRow).map(lambda x: int(x[1])).foreach(lambda x:accum.add(x))
assert(accum.value == 1300)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…